<a href="https://colab.research.google.com/github/Frederick-Numbisi/MappingDesertPlantAssemblages/blob/main/PCA_of_VegIndices_and_SpectralBands.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Colab Project for Calculating vegetation indices for AlUla Project

import ee
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize(project='tropvegclass') # Specify project

In [ ]:
!pip install earthengine-api #earth-engine Python API
!pip install geemap

In [ ]:
# Import the Folium library.
import folium
import ee

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)



In [ ]:
# install required libraries
!pip install -q rasterio
!pip install -q geopandas
!pip install git+https://github.com/tensorflow/examples.git
!pip install -U tfds-nightly
!pip install focal-loss

!pip install sentinelsat
!pip install folium
!apt install gdal-bin python-gdal python3-gdal
!apt install python3-rtree
!pip install git+git://github.com/geopandas/geopandas.git
!pip install descartes
!pip install wxee
!pip install patchify # install patchify tool into the environment

!pip install geotile
!pip install split-folders

In [ ]:
# import required libraries
import os, glob, functools, fnmatch
from zipfile import ZipFile
from itertools import product

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.grid'] = False
mpl.rcParams['figure.figsize'] = (12,12)

from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import pandas as pd
from PIL import Image

import rasterio
from rasterio import features, mask, windows

import geopandas as gpd

import tensorflow as tf
from tensorflow.python.keras import layers, losses, models
from tensorflow.python.keras import backend as K
#import tensorflow_addons as tfa

from tensorflow_examples.models.pix2pix import pix2pix

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from IPython.display import clear_output

from focal_loss import SparseCategoricalFocalLoss
from sklearn.metrics import confusion_matrix, f1_score

# Import the necessary libraries
from PIL import Image
from numpy import asarray

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# set your root directory to the shared drive folder
# root_dir = '/content/drive/Shared drives/servir-sat-ml/data/'
root_dir = '/content/drive/My Drive/'

images_Bands = "KSA_AlUla_SprectralBands_IQR" # Create dataset folder name

In [ ]:
#Al_Ula = ee.FeatureCollection("projects/ee-frednumbisi/assets/SAU_Al-Ula")

Al_Ula = ee.FeatureCollection("projects/ee-frednumbisi/assets/AlUla_AOIProject")

#Al_UlaROI = ee.Geometry.Polygon(ee.FeatureCollection("projects/ee-frednumbisi/assets/AlUla_BoundaryAOI"))

Al_UlaROI = Al_Ula.geometry()

In [ ]:
# set your root directory to your personal drive folder for file writes, so we don't overwrite each other
# We recommend you make a subfolder to keep your work.
#personal_dir = '/content/gdrive/My Drive/croptype/'

#personal_AlUla = '/content/drive/My Drive/RBGE_AlUla/AlUlaAOI/'

# We recommend you make a subfolder to keep your work.

#personal_Ashar = '/content/gdrive/My Drive/AsharLULCResults/'


#Refclass_dir = '/content/drive/My Drive/WAsharRefLULC2/'

#target_crs = 'epsg:32637' # UTM Zone 37N for KSA and AlUla


In [ ]:
# go to root directory
%cd $root_dir

/content/drive/My Drive


### Enabling GPU

This notebook can utilize a GPU and works better if you use one. Hopefully this notebook is using a GPU, and we can check with the following code.

If it's not using a GPU you can change your session/notebook to use a GPU.

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at: /device:GPU:0


### FASTER GPU

You can see what GPU you've been assigned at any time by executing the following cell. If the execution result of running the code cell below is 'Not connected to a GPU', you can change the runtime by going to <code>Runtime &gt; Change runtime type</code> in the menu to enable a GPU accelerator, and then re-execute the code cell.

In [ ]:
# Run Code to See Assigned GPUs
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Nov 22 14:17:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### More Memory Allocation

Users who have purchased one of Colab's paid plans have access to high-memory VMs when they are available.

See how much memory you have available at any time by running the following code cell. If the execution result of running the code cell below is 'Not using a high-RAM runtime', then you can enable a high-RAM runtime via <code>Runtime &gt; Change runtime type</code> in the menu. Then select High-RAM in the Runtime shape drop-down. After, re-execute the code cell.

In [ ]:
# Check Allocated Memory or RAM

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!ls -lah '/content/drive/My Drive/KSA_AlUla_SprectralBands_IQR'
#!ls -lah personal_dir

total 36G
-rw------- 1 root root 1.4G Nov 21 17:12 2023AlUla_IQR_image_B2max.tif
-rw------- 1 root root 2.9G Nov 21 08:14 2023AlUla_IQR_image_B2mean.tif
-rw------- 1 root root 1.4G Nov 21 13:11 2023AlUla_IQR_image_B2min.tif
-rw------- 1 root root 3.0G Nov 21 20:41 2023AlUla_IQR_image_B2std.tif
-rw------- 1 root root 1.4G Nov 21 16:29 2023AlUla_IQR_image_B3max.tif
-rw------- 1 root root 3.0G Nov 21 00:55 2023AlUla_IQR_image_B3mean.tif
-rw------- 1 root root 1.5G Nov 21 12:19 2023AlUla_IQR_image_B3min.tif
-rw------- 1 root root 3.1G Nov 21 19:47 2023AlUla_IQR_image_B3std.tif
-rw------- 1 root root 1.5G Nov 21 15:03 2023AlUla_IQR_image_B4max.tif
-rw------- 1 root root 3.0G Nov 20 23:35 2023AlUla_IQR_image_B4mean.tif
-rw------- 1 root root 1.6G Nov 21 10:52 2023AlUla_IQR_image_B4min.tif
-rw------- 1 root root 3.1G Nov 21 18:38 2023AlUla_IQR_image_B4std.tif
-rw------- 1 root root 1.6G Nov 21 17:51 2023AlUla_IQR_image_B8max.tif
-rw------- 1 root root 3.0G Nov 21 09:16 2023AlUla_IQR_image_B8m

In [ ]:
# Access the image files in directory
# Use the OpenCV library to read the image files in the "images" folder

import os
import cv2 # cv2 is already installed in the Colab Environment
from PIL import Image
import numpy as np
import rasterio
from patchify import patchify # to creat image patches to specific patch size
from sklearn.preprocessing import minmax_scale, StandardScaler, MinMaxScaler

from matplotlib import pyplot as plt

import re

In [ ]:
#%matplotlib inline

from sklearn.decomposition import PCA
import geopandas as gpd
import numpy as np
import pandas as pd
import cv2 as cv
from google.colab.patches import cv2_imshow
from osgeo import gdal
from sklearn.preprocessing import minmax_scale, StandardScaler, MinMaxScaler

import json
import wxee
import ee
import geemap
#import sklearn_flatten, sklearn_unflatten


In [ ]:
#minmaxscaler = MinMaxScaler()

In [ ]:

images_Bands = "KSA_AlUla_SprectralBands_IQR" # Create dataset folder name



imagesList = [] # create empty list

# Use the root directory folder and dataset folder names to source the image files
# use the os method to read the images in the folders
# use the "os.walk" method to select a particular folder: returns the paths, subdirectories and files for the selected folder

#for path, subdirs, files in os.walk(root_dir): # get information for the root directory
for path, subdirs, files in os.walk(os.path.join(root_dir, images_Bands)): # get information for the specified images directory
  dir_name = path.split(os.path.sep)[-1] # get the names of all directories in the root folder
  #print(dir_name)
  # if dir_name == 'Images': # can also use 'masks'
  StDevimages = os.listdir(path)# get the list of images in the path
  #print(StDevimages)
  print(len(StDevimages))
  for i, image_name in enumerate(StDevimages):
    if (image_name.endswith('.tif')): #can also specify '.png' if dir_names == 'masks'
      print(image_name) # enumerate and print each image name

      a = True # to avoid seeing the output





16
2023AlUla_IQR_image_B4mean.tif
2023AlUla_IQR_image_B3mean.tif
2023AlUla_IQR_image_B2mean.tif
2023AlUla_IQR_image_B8mean.tif
2023AlUla_IQR_image_B4min.tif
2023AlUla_IQR_image_B3min.tif
2023AlUla_IQR_image_B2min.tif
2023AlUla_IQR_image_B8min.tif
2023AlUla_IQR_image_B4max.tif
2023AlUla_IQR_image_B3max.tif
2023AlUla_IQR_image_B2max.tif
2023AlUla_IQR_image_B8max.tif
2023AlUla_IQR_image_B4std.tif
2023AlUla_IQR_image_B3std.tif
2023AlUla_IQR_image_B2std.tif
2023AlUla_IQR_image_B8std.tif


In [ ]:
!pip install sentinelsat
!pip install rasterio
!pip install folium
!apt install gdal-bin python-gdal python3-gdal
!apt install python3-rtree
!pip install git+git://github.com/geopandas/geopandas.git
!pip install descartes

In [ ]:
import folium
import os
import numpy as np

from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from shapely.geometry import MultiPolygon, Polygon
import rasterio as rio
from rasterio.plot import show
import rasterio.mask
import fiona

In [ ]:
# To Print All Files in the specified Directory that contain "IQR_image" in it name and has the extension .tif

import fnmatch
import os

for file in os.listdir(os.path.join(root_dir, images_Bands)):
  if fnmatch.fnmatch(file, '*IQR_image*.tif'):
    print(file)




2023AlUla_IQR_image_B4mean.tif
2023AlUla_IQR_image_B3mean.tif
2023AlUla_IQR_image_B2mean.tif
2023AlUla_IQR_image_B8mean.tif
2023AlUla_IQR_image_B4min.tif
2023AlUla_IQR_image_B3min.tif
2023AlUla_IQR_image_B2min.tif
2023AlUla_IQR_image_B8min.tif
2023AlUla_IQR_image_B4max.tif
2023AlUla_IQR_image_B3max.tif
2023AlUla_IQR_image_B2max.tif
2023AlUla_IQR_image_B8max.tif
2023AlUla_IQR_image_B4std.tif
2023AlUla_IQR_image_B3std.tif
2023AlUla_IQR_image_B2std.tif
2023AlUla_IQR_image_B8std.tif


Use Shapely Python library since our data is in Shapefiles and read it already as Geopandas GeodataFrame. (Note that if you have Geojson data, Sentinelsat provides a handy way to convert your data into a proper format in the query).

In [ ]:
# set variables for sentinel 2 timstamps and local coordinate reference system
#root_dir = './'

VIbandsIQR_Dir = 'KSA_AlUla_SprectralBands_IQR'

VIindex_IMGstamps = ['AlUla_VIstdDevIQR_stack.tif', 'AlUla_VImeanIQR_stack.tif']

test_sentinel_timestamp = ['2017-07-10']

target_crs2 = 'epsg:3857' #PseudoMercator

In [ ]:
# # CODE RUNS: commented or do not run to save RAM or Memory

# #stackStDevVI = cv2.imread(f'{stackedIQR_dir}/AlUla_VIstdDevIQR_stack.tif')
# stackStDevVI = rasterio.open(f'{stackedIQR_dir}/AlUla_VIstdDevIQR_stack.tif').read()

# stackStDevVI.shape

In [ ]:
# # Uncomment code block to run scripts: linked to 2 previous code blocks
# # Transpose the array to row, columns, bands

# stackStDevVIt = stackStDevVI.transpose(1,2,0)

# print(stackStDevVIt.shape)
# print(type(stackStDevVIt))

In [ ]:
# def RGBplot(image, bindex):  # band index should be in order of r,g,b
#   #assumes shape is as obtained above
#   img = np.dstack((image[:,:,bindex[0]]/np.percentile(image[:,:,bindex[0]],95),
#                   image[:,:,bindex[1]]/np.percentile(image[:,:,bindex[1]],95),
#                   image[:,:,bindex[2]]/np.percentile(image[:,:,bindex[2]],95)))
#   img = np.clip(img, 0, 1)
#   f = plt.figure()
#   plt.imshow(np.array(img))
#   plt.axis('off')
#   plt.show()

#Then create the loop to go through your images. For example,

# # how you access the folder will depend on your colab setup, naming system, etc
# for image in os.listdir(drive):
#   if image.endswith('.tif'):
#     [img, xsize, ysize, nbands] = loadTiff(os.path.join(drive,image))
#     RGBplot(img, [3,2,1])

In [ ]:
#RGBplot(stackStDevVI, [1,2,5])

Get the stacked images. Get the metadata and image for each Band representing the corresponding computed vegetation indices.

In [ ]:
#rasterio.open(f'{root_dir}/{images_StDev}/{image_name}') as src:
#      imgNM = src.read() # reads and opens each tiff image as an

# def VegIndex_read1(sentinel_timestamp): # images_StDev as example folder
#     vegindex_dir = os.path.join(root_dir,sentinel_timestamp)
#     #bands = glob.glob(sentinel_dir+'/**/*.tif',recursive=True)

#     src_grvi = rasterio.open(f'{vegindex_dir}/AlUla_VIstdDevIQR_stack.tif')
#     # array
#     arr_grvi = src_grvi.read()

#     # convert data type from float23 to 8bits unit (unit8)
#     # index_stack = (arr_grvi * 255).astype(np.uint8)

#     index_stack = arr_grvi.astype(np.uint8) # convert image type to array of type unit8
#     index_stack = np.where(np.isnan(index_stack), 0, index_stack) # if nan values, convert to zero; otherwise, retain value

#     # Transpose the array to row, columns, bands
#     index_stack = arr_grvi.transpose(1,2,0)

#     return vegindex_dir, index_stack, src_grvi

In [ ]:
# This is a trial code block used to create function in the following code block
# Uncomment to verify if code runs normally


vegindex_dir = os.path.join(root_dir,VIbandsIQR_Dir)
    #bands = glob.glob(vegindex_dir+'/**/*.tif',recursive=True)

imgDic = {} # Create dictionary to save images or their corresponding metadata

for path, subdirs, files in os.walk(vegindex_dir): # get information for the specified images directory
  dir_name = path.split(os.path.sep)[-1] # get the names of all directories in the root folder
  #print(dir_name)
  # if dir_name == 'Images': # can also use 'masks'
  StDevimages = os.listdir(path)# get the list of images in the path
  #print(StDevimages)

  # import fnmatch
  # import os

  # for file in os.listdir(os.path.join(root_dir, images_StDev)):
  #   if fnmatch.fnmatch(file, '*NDVI*.tif'):
  #     print(file)



  for image_name in StDevimages:
    #imgNM = cv2.imread(image_name)
      # if image_name.endswith(".tif"):
    # Read band metadata and arrays
    # metadata
    #src_2 = rasterio.open(fnmatch.fnmatch(image_name, '*NDVI*.tif')) #blue
    count = 1
    if fnmatch.fnmatch(image_name,'*IQR_image*.tif'):
      src_savi = rasterio.open(f'{vegindex_dir}/{image_name}')
      imgDic[count] = src_savi
      savi = imgDic[count].read()
      count += 1
      print(src_savi)
      print(image_name)
      print(imgDic)
      print(savi.shape)

In [ ]:
# DEFINE FUNCTION TO READ IMAGES IN TARGET DIRECTORY AND CREATE STACK OF SPECTRAL BANDS

import fnmatch
import os


def SpecBands_read(sentinel_timestamp): # images_StDev as example folder
    Bands_dir = os.path.join(root_dir,sentinel_timestamp)
    #bands = glob.glob(vegindex_dir+'/**/*.tif',recursive=True)

    #global src_sarvi, src_savi, src_ndvi, src_msavi, src_ipvi, src_gsavi, src_grvi

    globImages = {} # Create dictionary to hold VI(key) and the reference meta data for each image (Value)

    TranformedBands = []

    for path, subdirs, files in os.walk(Bands_dir): # get information for the specified images directory
      dir_name = path.split(os.path.sep)[-1] # get the names of all directories in the root folder
      #print(dir_name)
      # if dir_name == 'Images': # can also use 'masks'
      StDevimages = os.listdir(path)# get the list of images in the path
      #print(StDevimages)

      # import fnmatch
      # import os

      # for file in os.listdir(os.path.join(root_dir, images_StDev)):
      #   if fnmatch.fnmatch(file, '*NDVI*.tif'):
      #     print(file)



      #for i,image_name in enumerate(StDevimages):
      for image_name in StDevimages:

       #imgNM = cv2.imread(image_name)
         # if image_name.endswith(".tif"):
        # Read band metadata and arrays
        # metadata
        #src_2 = rasterio.open(fnmatch.fnmatch(image_name, '*NDVI*.tif')) #blue
        #count = 1
        if fnmatch.fnmatch(image_name,'*image_B4mean.tif'):
          src_B4mean = rasterio.open(f'{Bands_dir}/{image_name}')
          globImages['B4mean'] = src_B4mean
          #count += 1
        elif fnmatch.fnmatch(image_name,'*image_B3mean.tif'): # Provided specific prefixes
          src_B3mean = rasterio.open(f'{Bands_dir}/{image_name}') #B3mean
          globImages['B3mean'] = src_B3mean
        elif fnmatch.fnmatch(image_name,'*image_B2mean.tif'):
          src_B2mean = rasterio.open(f'{Bands_dir}/{image_name}') #B2mean
          globImages['B2mean'] = src_B2mean
        elif fnmatch.fnmatch(image_name,'*image_B8mean.tif'):
          src_B8mean = rasterio.open(f'{Bands_dir}/{image_name}') #B8mean
          globImages['B8mean'] = src_B8mean

        elif fnmatch.fnmatch(image_name,'*image_B4std.tif'):
          src_B4std = rasterio.open(f'{Bands_dir}/{image_name}') #
          globImages['B4std'] = src_B4std
        elif fnmatch.fnmatch(image_name,'*image_B3std.tif'):
          src_B3std = rasterio.open(f'{Bands_dir}/{image_name}') #
          globImages['B3std'] = src_B3std
        elif fnmatch.fnmatch(image_name,'*image_B2std.tif'):
          src_B2std = rasterio.open(f'{Bands_dir}/{image_name}') #
          globImages['B2std'] = src_B2std
        elif fnmatch.fnmatch(image_name,'*image_B8std.tif'):
          src_B8std = rasterio.open(f'{Bands_dir}/{image_name}') #
          globImages['B8std'] = src_B8std

        elif fnmatch.fnmatch(image_name,'*image_B4min.tif'):
          src_B4min = rasterio.open(f'{Bands_dir}/{image_name}') #B4minn
          globImages['B4min'] = src_B4min
        elif fnmatch.fnmatch(image_name,'*image_B3min.tif'):
          src_B3min = rasterio.open(f'{Bands_dir}/{image_name}') #B3minn
          globImages['B3min'] = src_B3min
        elif fnmatch.fnmatch(image_name,'*image_B2min.tif'):
          src_B2min = rasterio.open(f'{Bands_dir}/{image_name}') #B2minn
          globImages['B2min'] = src_B2min
        elif fnmatch.fnmatch(image_name,'*image_B8min.tif'):
          src_B8min = rasterio.open(f'{Bands_dir}/{image_name}') #B8minn
          globImages['B8min'] = src_B8min

        elif fnmatch.fnmatch(image_name,'*image_B4max.tif'):
          src_B4max = rasterio.open(f'{Bands_dir}/{image_name}') #
          globImages['B4max'] = src_B4max
        elif fnmatch.fnmatch(image_name,'*image_B3max.tif'):
          src_B3max = rasterio.open(f'{Bands_dir}/{image_name}') #
          globImages['B3max'] = src_B3max
        elif fnmatch.fnmatch(image_name,'*image_B2max.tif'):
          src_B2max = rasterio.open(f'{Bands_dir}/{image_name}') #
          globImages['B2max'] = src_B2max
        elif fnmatch.fnmatch(image_name,'*image_B8max.tif'):
          src_B8max = rasterio.open(f'{Bands_dir}/{image_name}') #
          globImages['B8max'] = src_B8max


    # Read image using the VI "Key" to reference the metadata
    arr_img1 = globImages['B4mean'].read()
    arr_img2 = globImages['B3mean'].read()
    arr_img3 = globImages['B2mean'].read()
    arr_img4 = globImages['B8mean'].read()
    arr_img5 = globImages['B4std'].read()
    arr_img6 = globImages['B3std'].read()
    arr_img7 = globImages['B2std'].read()
    arr_img8 = globImages['B8std'].read()

    arr_img9 = globImages['B4min'].read()
    arr_img10 = globImages['B3min'].read()
    arr_img11 = globImages['B2min'].read()
    arr_img12 = globImages['B8min'].read()
    arr_img13 = globImages['B4max'].read()
    arr_img14 = globImages['B3max'].read()
    arr_img15 = globImages['B2max'].read()
    arr_img16 = globImages['B8max'].read()


    # Covert from float32 to uint8 and Transpose the array to row, columns, bands
    tanspIMG1 = arr_img1.astype(np.uint8)#.transpose(1,2,0)
    tanspIMG2 = arr_img2.astype(np.uint8)#.transpose(1,2,0)
    tanspIMG3 = arr_img3.astype(np.uint8)#.transpose(1,2,0)
    tanspIMG4 = arr_img4.astype(np.uint8)#.transpose(1,2,0)
    tanspIMG5 = arr_img5.astype(np.uint8)#.transpose(1,2,0)
    tanspIMG6 = arr_img6.astype(np.uint8)#.transpose(1,2,0)
    tanspIMG7 = arr_img7.astype(np.uint8)#.transpose(1,2,0)
    tanspIMG8 = arr_img8.astype(np.uint8)#.transpose(1,2,0)

    tanspIMG9 = arr_img9.astype(np.uint8)#.transpose(1,2,0)
    tanspIMG10 = arr_img10.astype(np.uint8)#.transpose(1,2,0)
    tanspIMG11 = arr_img11.astype(np.uint8)#.transpose(1,2,0)
    tanspIMG12 = arr_img12.astype(np.uint8)#.transpose(1,2,0)
    tanspIMG13 = arr_img13.astype(np.uint8)#.transpose(1,2,0)
    tanspIMG14 = arr_img14.astype(np.uint8)#.transpose(1,2,0)
    tanspIMG15 = arr_img15.astype(np.uint8)#.transpose(1,2,0)
    tanspIMG16 = arr_img16.astype(np.uint8)#.transpose(1,2,0)




    index_stack1 = np.dstack((tanspIMG1, tanspIMG2, tanspIMG3, tanspIMG4))
    #index_stack2 = np.dstack((tanspIMG4, tanspIMG5, tanspIMG6, tanspIMG7, tanspIMG8))

    #index_stack3 = np.dstack((tanspIMG9, tanspIMG10, tanspIMG11, tanspIMG12))
    #index_stack4 = np.dstack((tanspIMG13, tanspIMG14, tanspIMG15, tanspIMG16))

    return index_stack1
    #return index_stack1, index_stack2

In [ ]:
# Run function using folder containing StdDevIQR images
# Create 2 rasterstacks: 3 Bands (GRIV, IPVI and MSAVI) and 4 Bands (GSAVI, NDVI, SAVI, SARVI)
IQR_bandstack = SpecBands_read(VIbandsIQR_Dir)
print(IQR_bandstack.shape)
print(type(IQR_bandstack))


##**Principal Component Analysis (PCA)**
PCA is a very useful technique in improving your supervised classification results. This is a statistical technique that compresses data from a large number of bands into fewer uncorrelated bands. You can run PCA on your image and add the first few (typically 3) principal component bands to the original composite before sampling training points. The variance from the original multi-spectral (e.g. 28-band) image is captured in the 3-band (or 5-band) PCA image. This sends a stronger signal to the classifier and improves accuracy by allowing it to distinguish different classes better.

**COMPUTE PCA OF StdDEV OF SPECTRAL BANDS**

In [ ]:
# RESHAPING THE DIMENSION

# Get the for of the stacked images as array
n_samples, nx, ny = IQR_bandstack.shape

# Reshape the data from 3D to 2D
Bandstack_arr_2D = IQR_bandstack.reshape(n_samples, nx*ny)

In [ ]:
# DETERMINE THE COMPONENT, STANDARDISE THE VALUES AND FIT TO PCA MODEL

# Determine the number of components
pcaAnalysis = PCA(n_components = 5)

# Scale the 2D data
Bandstack_arr_2D_Scaled = MinMaxScaler().fit_transform(Bandstack_arr_2D)

# Run the PCA
pcaAnalysis.fit(Bandstack_arr_2D_Scaled)

##Calculate the Covariance, Eigenvalues, Eigenvectors and the Variance-ratio

In [ ]:
# Compute Covariance Matrix
Covr_Matrix = np.cov(Bandstack_arr_2D_Scaled)

# Get the Eigenvalue and Eigenvector
Eigen_values, Eigen_vectors = np.linalg.eig(Covr_Matrix)

# Get the Variance Ratio
var_Ratio = pcaAnalysis.explained_variance_ratio_

# Get Variance ratio in percentage
varRatio_Perc = []
for i in var_Ratio:
  percent = (i * 100).round(3)
  varRatio_Perc.append(percent)

##Identify Band loadings or contribution to components

In [ ]:
# Compunte EigenVector Dataframe
EigenVectors_Df = pd.DataFrame(data = Eigen_vectors,
                               columns = ['Component1', 'Component2', 'Component3', 'Component4', 'Component5'],
                               index = ['Band1', 'Band2', 'Band3', 'Band4', 'Band5', 'Band6', 'Band7', 'Band8'])
                              #index = ['Band9', 'Band10', 'Band11', 'Band12', 'Band13', 'Band14', 'Band15', 'Band16'])

# Get Eigen vector (contribution of all bands)
EigenVectors_Df.round(3)

#Extract, Visualise and Export PCA Components as Images

In [ ]:
# Fit the pca to scaled data
pca_scaled_img = pcaAnalysis.fit_transform(Bandstack_arr_2D_Scaled)

# Inverse Component to Original 2D Space
image_inversed = pcaAnalysis.inverse_transform(pca_scaled_img)
image_scaled = MinMaxScaler(feature_range=(0, 1), copy=True).fit_transform(image_inversed)

# Reshape PCA Components back to 3D
PCAcomponents = image_scaled.reshape((n_samples, nx, ny))

# Verify the Dimensions
PCAcomponents.shape, image_inversed.shape


**Extract the PCA Image Components**

In [ ]:
PCA_1 = PCAcomponents[0]
PCA_2 = PCAcomponents[1]
PCA_3 = PCAcomponents[2]
PCA_4 = PCAcomponents[3]
PCA_5 = PCAcomponents[4]

In [ ]:
PCA_imageBandsIQR2 = { "image_PCA1":PCA_1, "image_PCA2":PCA_2, "image_PCA3":PCA_3}

In [ ]:
# Export the Spectral Band IQR Images
for key, value in PCA_imageBandsIQR2.items():
  fname = ee.String(key).getInfo()

  task = ee.batch.Export.image.toDrive(
    #image = Bandname.replace('"', ''),
    #image = Bandname.strip(),
    image = value,
    region = Al_Ula.geometry().bounds(), # Or use custom ee.Geometry.Rectangle([minlon, minlat, maxlon, maxlat])
    description = '2023AlUla_'+fname,
    scale = 10,
    folder = "KSA_AlUla_PCABands",
    crs = 'EPSG:4326',
    #crs ='EPSG:3857', # UTM CRS for Saudi
    #crsTransform = proj["transform"],
    maxPixels = 1e13,
    fileFormat = "GeoTIFF",
    formatOptions = {
    'cloudOptimized': True
  }
    )

  task.start()

  import time

  while task.active():
    counter = 0
    print('Polling for task (id: {}).process: {}'.format(task.id, counter))
    counter += 1
    time.sleep(5)


##Create Stack and Export to Google Drive

In [ ]:
# Open the first band rasteR
# Stacking multiple .tif files into a multiple band stack using Rasterio
# Using Rasterio to stack multiple bands without using subprocess commands
#

import rasterio

#file_list = ['file1.tif', 'file2.tif', 'file3.tif']

# Read metadata of first file
with rasterio.open(VIband_rasters[0]) as src0:
    meta = src0.meta

# Update meta to reflect the number of layers
meta.update(count = len(VIband_rasters))

# Read each layer and write it to stack
with rasterio.open('VIBandsstack.tif', 'w', **meta) as dst:
    for id, layer in enumerate(VIband_rasters, start=1):
        with rasterio.open(layer) as src1:
            dst.write_band(id, src1.read(1))

In [ ]:
# Get location of the band rasters images (.tif files) in Google Drive
VIbands_folder_path = "/content/drive/My Drive/KSA_AlUla_SprectralBands_IQR/"

# Get the names of all band rasters in the folder as a list

VIband_rasters = [os.path.join(VIbands_folder_path, f) for f in os.listdir(VIbands_folder_path) if f.endswith(".tif")]


In [ ]:
# Export the image, specifying scale and region.
# Export the Standardised IQR Images

task = ee.batch.Export.image.toDrive(
    image = compositeMeanVIndices,
    region = Al_Ula.geometry().bounds(), # Or use custom ee.Geometry.Rectangle([minlon, minlat, maxlon, maxlat])
    description = '2023AlUla_MeanVI_PCA',
    scale = 10,
    folder = "KSA_AlUla_VIIQR_PCAbands",
    crs = 'EPSG:4326',
    #crs ='EPSG:3857', # UTM CRS for Saudi
    #crsTransform = proj["transform"],
    maxPixels = 1e13,
    fileFormat = "GeoTIFF",
    formatOptions = {
    'cloudOptimized': True
  }
    )

task.start()

import time
while task.active():
  print('Polling for task (id: {}).'.format(task.id))
  time.sleep(5)